In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from TrajectoryDataset import TrajectoryDataset
from DLModels import GraphTrajectoryLSTM
from Trainer import Trainer
from visualizer import visualize_predictions
import pickle
import os

def load_sequences(folder_path):
    all_sequences = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.pkl'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as f:
                sequences = pickle.load(f)
                all_sequences.extend(sequences)
    print(f"Loaded {len(all_sequences)} sequences")
    return all_sequences

def train():
    # Hyperparameters
    input_sizes = {
        'node_features': 4,
        'position': 2,
        'velocity': 2,
        'steering': 1,
        'object_in_path': 1,
        'traffic_light_detected': 1
    }
    hidden_size = 128
    num_layers = 2
    input_seq_len = 3  # past trajectory length
    output_seq_len = 3  # future prediction length
    batch_size = 128
    num_epochs = 50
    learning_rate = 0.001
    
    # Data loading
    data_folder = "Dataset/Sequence_Dataset"
    dataset = TrajectoryDataset(data_folder)
    
    # Split the dataset
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    print(f"Train set size: {len(train_dataset)}")
    print(f"Test set size: {len(test_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)
    
    # Model initialization
    model = GraphTrajectoryLSTM(input_sizes, hidden_size, num_layers, input_seq_len, output_seq_len)
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainer = Trainer(model, train_loader, test_loader, learning_rate, device)
    trained_model = trainer.train(num_epochs)
    
    # Save the trained model
    torch.save(trained_model.state_dict(), "graph_trajectory_model_gaussian.pth")
    
    # Evaluate on test set
    test_loss = trainer.validate()
    print(f"Test Loss: {test_loss:.4f}")

    # Visualization
    all_sequences = load_sequences(data_folder)
    visualize_predictions(trained_model, dataset, device, all_sequences)

def collate_fn(batch):
    past_batch = {k: torch.stack([item[0][k] for item in batch]) for k in batch[0][0].keys()}
    future_batch = {k: torch.stack([item[1][k] for item in batch]) for k in batch[0][1].keys()}
    graph_batch = {
        'node_features': torch.stack([item[2]['node_features'] for item in batch]),
        'adj_matrix': torch.stack([item[2]['adj_matrix'] for item in batch])
    }
    return past_batch, future_batch, graph_batch

train()

Loaded 5661 sequences
Train set size: 4528
Test set size: 1133


Training: 100%|██████████| 36/36 [00:10<00:00,  3.51it/s]


Epoch 1/100, Train Loss: 1694.0852, Val Loss: 1064.3019


Training: 100%|██████████| 36/36 [00:10<00:00,  3.56it/s]


Epoch 2/100, Train Loss: 886.1770, Val Loss: 723.0309


...

Training: 100%|██████████| 36/36 [00:09<00:00,  3.64it/s]


Epoch 66/100, Train Loss: -1382.4530, Val Loss: -1246.2598


Training: 100%|██████████| 36/36 [00:09<00:00,  3.68it/s]


Epoch 67/100, Train Loss: -1371.0446, Val Loss: -1393.5069


Training:   3%|▎         | 1/36 [00:00<00:09,  3.67it/s]